# Projet de Machine Learning for data sciences
Utilisation des données internet sur les articles publiés sur https://news.bitcoin.com/ et les données marché pour prédire la montée ou la descente du BitCoin.  

## Import des packages
Les fonctions créées pour le projet seront expliquée lors de l'utilisation.

In [12]:
import pandas as pd #pandas
import web_knwoledge.api_bitcoin as bitcoin_api #data making
from importlib import reload

Il faut lancer web_knowledge.scrapper sur l'invite de commande (ou utiliser les données déjà récoltées). 
        ==> dossier wayback_dowloads
> python web_knowledge/scrapper.py 'https://news.bitcoin.com'

In [13]:
#Récupération des données de binance :
reload(bitcoin_api)
data = bitcoin_api.get_data('1w') # On est limité à un interval de 1 semaine pour s'accorder aux données web

## Données disponibles grâce à l'appel API binance :
- Open time: The timestamp (in milliseconds) indicating when the candlestick period starts.
- Open: The opening price of the asset for the candlestick period.
- High: The highest price of the asset during the candlestick period.
- Low: The lowest price of the asset during the candlestick period.
- Close: The closing price of the asset for the candlestick period.
- Volume: The trading volume of the asset for the candlestick period.
- Close time: The timestamp (in milliseconds) indicating when the candlestick period ends.
- Quote asset volume: The trading volume of the quote asset for the candlestick period.
- Number of trades: The number of trades that occurred during the candlestick period.
- Taker buy base asset volume: The volume of the base asset involved in taker trades during the candlestick period.
- Taker buy quote asset volume: The volume of the quote asset involved in taker trades during the candlestick period.
- Ignore: This column is typically ignored and contains no relevant data.

In [14]:
columns=["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume","Number of trades","Taker buy base","Taker buy quote","Ignore"]
Binance_dataframe=pd.DataFrame(data,columns=columns)

### On va maintenant organiser nos données pour permettre la jointure avec les données web

In [92]:
from datetime import datetime,timedelta

In [16]:
open_time=[datetime.strftime(datetime.fromtimestamp(i/1000),format='%Y%m%d') for i in Binance_dataframe["Open time"]]
Binance_dataframe["Open time"]=open_time

In [17]:
#Sauvegarde des données
Binance_dataframe.to_csv("data/market/Binance_data.csv",index=False)

### On va maintenant convertir nos fichiers html en txt (formaté)

In [18]:
from data_processing.from_html_to_txt import extract_content_from_html as ext_html
import os

In [19]:
files = os.listdir("wayback_downloads")
log=''
for i in files:
    already_done = os.listdir("data/html/")
    if f'{i.split(".")[0][:8]}.txt' not in already_done:
        try:
            ext_html("wayback_downloads/" + i, ['div', 'p', 'h6', 'h4'], f'data/html/{i.split(".")[0][:8]}.txt')
        except Exception as e:
            log+=f"Failed to process {i}: {e}\n"
            with open("log.txt","w") as f: f.write(log)

## Création du dataframe pour ACM
On utilise nos fichiers HTML formaté en txt pour compter les mots

In [20]:
from data_processing.text_processing import proc_make_csv as make_csv

Ici on lemmatise notre texte pour qu'il ne contienne que les mots importants (pas générique) du texte.
Réutilisation d'un script fait en Analyse textuelle

In [24]:
word_count=make_csv()
word_count.index = [i[:8] for i in os.listdir("data/html/")]

In [26]:
word_count.to_csv("data/word_count.csv")

In [30]:
word_count=pd.read_csv("data/word_count.csv",index_col=0)
word_count=word_count.T

In [63]:
cache=word_count.iloc[0].value_counts()
cache=pd.DataFrame(data=cache.to_numpy().flatten(),index=cache.index.to_numpy().flatten(),columns=["count"])
int(cache.iloc[0])

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13264\4113805289.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  int(cache.iloc[0])


99

In [64]:
to_drop=[]
for i in range(len(word_count.index)):
    cache=word_count.iloc[i].value_counts()
    cache=pd.DataFrame(data=cache.to_numpy().flatten(),index=cache.index.to_numpy().flatten(),columns=["count"])
    if 0 not in cache.index:
        to_drop.append(word_count.index[i])
    elif int(cache.iloc[0])>len(word_count.columns)*0.5:
        to_drop.append(word_count.index[i])

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13264\3963066350.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  elif int(cache.iloc[0])>len(word_count.columns)*0.5:


> len(word_count.index)-len(to_drop)
>>175

On garde alors 175 mots sur les plus de 9000 existants ==> Analyse globale permettant clustering

In [73]:
word_count.drop(to_drop,inplace=True)
word_count.to_csv("data/word_count.csv") #check point
word_count=word_count.T

On formate les données extraite des pages html pour n'avoir qu'un enregistrement par semaine pour s'accorder aux données de marché.

In [103]:
for i in Binance_dataframe['Open time']:
    borne_inf=int(datetime.strftime(datetime.strptime(Binance_dataframe["Open time"][i],'%Y%m%d') - timedelta(days=7),format='%Y%m%d'))
    borne_sup=int(datetime.strftime(datetime.strptime(Binance_dataframe["Open time"][i],'%Y%m%d'),format='%Y%m%d'))
    word_count=word_count.index[(word_count.index>=borne_inf) & (word_count.index<=borne_sup)]

In [ ]:
word_count=pd.read_csv("data/word_count.csv",index_col=0)
word_count=word_count.T
word_count.index=[int(i) for i in word_count.index]
index_to_keep=[]
for i in range(len(Binance_dataframe['Open time'])):
    borne=int(datetime.strftime(datetime.strptime(Binance_dataframe["Open time"][i],'%Y%m%d'),format='%Y%m%d'))
    cache=max(word_count.index[(word_count.index<=borne)])
    if cache not in index_to_keep:
        index_to_keep.append(cache)